In [2]:
pip install google-cloud-video-transcoder

     |████████████████████████████████| 61 kB 631 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 1.4 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [192]:
from google.cloud.video import transcoder
from google.protobuf.duration_pb2 import Duration

In [193]:

parent_location ='projects/320823659895/locations/europe-west1'


transcoder_client = transcoder.TranscoderServiceClient()

In [195]:
%%writefile function_code/Transcoder_utils.py

from google.cloud.video import transcoder
from google.protobuf.duration_pb2 import Duration


OVERLAY_IMAGE_URI = 'gs://scary-videos/no_see.jpg'

# create overlay image object
no_see_image = transcoder.Overlay.Image(
    uri = OVERLAY_IMAGE_URI, 
    alpha=1, 
    resolution= transcoder.Overlay.NormalizedCoordinate(x=1., y=1.)
)


def get_nanos_from_seconds(seconds):
    return int ((seconds - int(seconds)) / 1e-9) 

def create_static_overlay_segment(start_time_seconds, end_time_seconds):
    
    animation_start = transcoder.Overlay.Animation()
    animation_start.animation_static = transcoder.Overlay.AnimationStatic()
    animation_start.animation_static.start_time_offset = Duration(seconds = int(start_time_seconds), 
                                                                  nanos = get_nanos_from_seconds(start_time_seconds) )
    animation_start.animation_static.xy = transcoder.Overlay.NormalizedCoordinate(x=0., y=0.)
    animation_start
    
    animation_end = transcoder.Overlay.Animation()
    animation_end.animation_end = transcoder.Overlay.AnimationEnd()
    animation_end.animation_end.start_time_offset = Duration(seconds = int(end_time_seconds), 
                                                             nanos = get_nanos_from_seconds(end_time_seconds))
    animation_end
    
    return [animation_start, animation_end]


def create_fade_overlay_segment(start_time_seconds, end_time_seconds, fade_duration = 0.5):
    
    # create fade in animation     
    animation_start = transcoder.Overlay.Animation()
    animation_start.animation_fade = transcoder.Overlay.AnimationFade()
    
    animation_start.animation_fade.start_time_offset = Duration(seconds = int(start_time_seconds - fade_duration), 
                                                                nanos = get_nanos_from_seconds(start_time_seconds - fade_duration) )
    
    animation_start.animation_fade.end_time_offset = Duration(seconds = int(start_time_seconds ), 
                                                                nanos = get_nanos_from_seconds(start_time_seconds))
    
    animation_start.animation_fade.fade_type = transcoder.Overlay.FadeType.FADE_IN
    animation_start.animation_fade.xy = transcoder.Overlay.NormalizedCoordinate(x=0., y=0.)
    
    
    # create fade out animation  
    animation_end = transcoder.Overlay.Animation()
    animation_end.animation_fade = transcoder.Overlay.AnimationFade()
    animation_end.animation_fade.start_time_offset = Duration(seconds = int(end_time_seconds), 
                                                              nanos = get_nanos_from_seconds(end_time_seconds))
    
    animation_end.animation_fade.end_time_offset = Duration(seconds = int(end_time_seconds + fade_duration), 
                                                            nanos = get_nanos_from_seconds(end_time_seconds + fade_duration))
    animation_end.animation_fade.fade_type = transcoder.Overlay.FadeType.FADE_OUT
    
    animation_end.animation_fade.xy = transcoder.Overlay.NormalizedCoordinate(x=0., y=0.)
    
    
    return [animation_start, animation_end]


def create_video_overlay_job(input_url, output_folder, output_file_name, overlay_segments):

    # creaet overlays     
    overlay = transcoder.Overlay(image = no_see_image)
    overlay.animations = overlay_segments
    
    # create job configuration     
    job_config = transcoder.JobConfig()
    job_config.overlays = [overlay]
    
    # get video stream     
    elementary_video_stream = transcoder.ElementaryStream()
    elementary_video_stream.key = 'video1'
    elementary_video_stream.video_stream = transcoder.VideoStream(frame_rate=30, bitrate_bps=550000)
    
    # get audio stream     
    elementary_audio_stream = transcoder.ElementaryStream()
    elementary_audio_stream.key = 'audio1'
    elementary_audio_stream.audio_stream = transcoder.AudioStream(bitrate_bps=64000)
    
    # include video and audio streams     
    job_config.elementary_streams = [elementary_video_stream, elementary_audio_stream]
   
    # create output video + audio     
    mux_stream = transcoder.MuxStream(
        key = output_file_name,
        elementary_streams = ['video1', 'audio1']
    )
    job_config.mux_streams = [mux_stream]
    
    # create job     
    job = transcoder.Job()
    job.input_uri = input_url
    job.output_uri = output_folder
    job.config = job_config
    
    return job
    

Writing function_code/Transcoder_utils.py


In [185]:
job = create_video_overlay_job(input_url='gs://scary-videos/beyond-the-map-rio.mp4', 
                               output_folder='gs://scary-videos-fixed/', 
                               output_file_name='def_test3🙈', 
                               overlay_segments= create_overlay_segment(2.5, 9) + create_fade_overlay_segment(14.6, 19.5) + create_fade_overlay_segment(24.6, 29.5))

job_request = transcoder.CreateJobRequest(parent = parent_location, job = job)

In [186]:
transcoder_client.create_job(job_request)

name: "projects/320823659895/locations/europe-west1/jobs/feb0022f33eeefabd1ffe9ebf7f1fe18"
config {
  inputs {
    key: "input0"
    uri: "gs://scary-videos/beyond-the-map-rio.mp4"
  }
  edit_list {
    key: "atom0"
    inputs: "input0"
    start_time_offset {
    }
  }
  elementary_streams {
    video_stream {
      codec: "h264"
      profile: "high"
      preset: "veryfast"
      pixel_format: "yuv420p"
      bitrate_bps: 550000
      rate_control_mode: "vbr"
      crf_level: 21
      vbv_size_bits: 550000
      vbv_fullness_bits: 495000
      gop_duration {
        seconds: 3
      }
      entropy_coder: "cabac"
      frame_rate: 30.0
    }
    key: "video1"
  }
  elementary_streams {
    audio_stream {
      codec: "aac"
      bitrate_bps: 64000
      channel_count: 2
      channel_layout: "fl"
      channel_layout: "fr"
      sample_rate_hertz: 48000
    }
    key: "audio1"
  }
  mux_streams {
    key: "def_test3\360\237\231\210"
    file_name: "def_test3\360\237\231\210.mp4"
   

In [179]:
transcoder_client.get_job(name = parent_location +'/jobs/'+ 'e440b88b5d3b64c757d699847837900f')

name: "projects/320823659895/locations/europe-west1/jobs/e440b88b5d3b64c757d699847837900f"
config {
  inputs {
    key: "input0"
    uri: "gs://scary-videos/beyond-the-map-rio.mp4"
  }
  edit_list {
    key: "atom0"
    inputs: "input0"
    start_time_offset {
    }
  }
  elementary_streams {
    video_stream {
      codec: "h264"
      profile: "high"
      preset: "veryfast"
      pixel_format: "yuv420p"
      bitrate_bps: 550000
      rate_control_mode: "vbr"
      crf_level: 21
      vbv_size_bits: 550000
      vbv_fullness_bits: 495000
      gop_duration {
        seconds: 3
      }
      entropy_coder: "cabac"
      frame_rate: 30.0
    }
    key: "video1"
  }
  mux_streams {
    key: "safe_video"
    file_name: "safe_video.mp4"
    container: "mp4"
    elementary_streams: "video1"
  }
  output {
    uri: "gs://scary-videos-fixed/"
  }
  overlays {
    image {
      uri: "gs://scary-videos/no_see.jpg"
      resolution {
        x: 1.0
        y: 1.0
      }
      alpha: 1.0
    }

In [180]:
# transcoder_client.list_jobs(parent=parent_location)

In [ ]:
{
  "config": {
    "inputs": [
          {
            "key": "input0",
            "uri": "gs://scary-videos/5-generation-bakers.mp4"
          }
        ],
    "elementaryStreams": [
      {
        "key": "video-stream0",
        "videoStream": {
          "codec": "h264",
          "heightPixels": 360,
          "widthPixels": 640,
          "bitrateBps": 550000,
          "frameRate": 60
        }
      },
      {
        "key": "audio-stream0",
        "audioStream": {
          "codec": "aac",
          "bitrateBps": 64000
        }
      }
    ],
    "muxStreams": [
      {
        "key": "sd",
        "container": "mp4",
        "elementaryStreams": [
          "video-stream0",
          "audio-stream0"
        ]
      }
    ],
    "output": {
      "uri": "gs://scary-videos/output/"
    },
    "overlays": [
      {
        "image": {
          "uri": "gs://scary-videos/no_see.jpg",
          "resolution": {
            "x": 1,
            "y": 1
          },
          "alpha": 1
        },
        "animations": [
          {
            "animationStatic": {
              "xy": {
                "x": 0,
                "y": 0
              },
              "startTimeOffset": "5s"
            }
          },
          {
            "animationEnd": {
              "startTimeOffset": "10s"
            }
          },          {
            "animationStatic": {
              "xy": {
                "x": 0,
                "y": 0
              },
              "startTimeOffset": "15s"
            }
          },
          {
            "animationEnd": {
              "startTimeOffset": "20s"
            }
          }

        ]
      }
    ]
  }
}